## Oppgave 2 a)

In [1]:
import numpy as np
kb = 1.38 * 10**-23
T = 298
beta = 1/(kb*T)

def V21(x):
    return 1/beta
def V22(x):
    return -x/beta

def walkDirection(pMinus,pPluss):
    tilfeldigVerdi = np.random.ranf()
    if tilfeldigVerdi<=pMinus:
        return -1
    elif tilfeldigVerdi >=(1-pPluss):
        return 1
    else:
        return 0



def sannsynligheter(beta,V,x):
    pMinus = 1/(1+np.exp(-beta*(V(x-1)-V(x+1)))+np.exp(-beta*(V(x)-V(x+1))))
    pPluss = 1/(1+np.exp(-beta*(V(x+1)-V(x-1)))+np.exp(-beta*(V(x)-V(x-1))))
    return pMinus, pPluss


def simulasjon(steg, beta, V, antallPartikler, mellomrom):
    posisjoner = np.arange(1,(antallPartikler+1)*(mellomrom+1)+1,mellomrom+1)
    for i in range(steg):
        for j in range(len(posisjoner)):
            pMinus, pPluss = sannsynligheter(beta,V,posisjoner[j])
            skritt =  walkDirection(pMinus,pPluss)
            #if not ((posisjoner[j] + skritt) in posisjoner):
                #posisjoner[i]+=skritt
            posisjoner[j]+=skritt
    return posisjoner
print(simulasjon(100,beta,V21,101,2))

[  7   8   5   7   7   5  25  25  40  34  35  23  32  40  44  40  39  46
  57  65  52  72  64  69  75  72  74  79  79  79 102  88 103  97  96 106
  89 105 126 124 112 112 123 130 137 133 139 140 143 159 153 147 151 179
 162 168 175 173 167 181 183 187 178 194 196 196 200 205 214 213 211 207
 214 226 226 224 231 231 237 224 227 239 251 258 249 239 257 257 266 273
 279 265 283 283 286 281 292 294 291 317 305 307]
